In [ ]:
#!kill -9 -1

In [1]:
from itertools import chain
import random
import numpy as np
import pandas as pd
import sklearn as sk
import sklearn.metrics
import scipy as sp
import sklearn.linear_model
import tensorflow as tf
sess = tf.Session()
from keras import backend as K
from keras import *
from keras.layers import *
from keras.callbacks import LambdaCallback, Callback, ReduceLROnPlateau, TensorBoard
import sklearn.linear_model
#!pip install hyperas
from hyperas.distributions import uniform, choice
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
K.set_session(sess)

Using TensorFlow backend.


In [2]:
"""
Created on Tue Nov  6 10:06:52 2018

@author: yandexdataschool

Original Code found in:
https://github.com/yandexdataschool/roc_comparison

updated: Raul Sanchez-Vazquez
"""

import scipy.stats
from scipy import stats

# AUC comparison adapted from
# https://github.com/Netflix/vmaf/
def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=np.float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def compute_midrank_weight(x, sample_weight):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    cumulative_weight = np.cumsum(sample_weight[J])
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = cumulative_weight[i:j].mean()
        i = j
    T2 = np.empty(N, dtype=np.float)
    T2[J] = T
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count, sample_weight):
    if sample_weight is None:
        return fastDeLong_no_weights(predictions_sorted_transposed, label_1_count)
    else:
        return fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight)


def fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank_weight(positive_examples[r, :], sample_weight[:m])
        ty[r, :] = compute_midrank_weight(negative_examples[r, :], sample_weight[m:])
        tz[r, :] = compute_midrank_weight(predictions_sorted_transposed[r, :], sample_weight)
    total_positive_weights = sample_weight[:m].sum()
    total_negative_weights = sample_weight[m:].sum()
    pair_weights = np.dot(sample_weight[:m, np.newaxis], sample_weight[np.newaxis, m:])
    total_pair_weights = pair_weights.sum()
    aucs = (sample_weight[:m]*(tz[:, :m] - tx)).sum(axis=1) / total_pair_weights
    v01 = (tz[:, :m] - tx[:, :]) / total_negative_weights
    v10 = 1. - (tz[:, m:] - ty[:, :]) / total_positive_weights
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def fastDeLong_no_weights(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating
              Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def compute_ground_truth_statistics(ground_truth, sample_weight):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    if sample_weight is None:
        ordered_sample_weight = None
    else:
        ordered_sample_weight = sample_weight[order]

    return order, label_1_count, ordered_sample_weight


def delong_roc_variance(ground_truth, predictions, sample_weight=None):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(
        ground_truth, sample_weight)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count, ordered_sample_weight)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov


In [3]:
def data():

    
    import pandas as pd

    column_names = ['age',
                'sex',
                'cp',
                'trestbps',
                'chol',
                'fbs',
                'restecg',
                'thalach',
                'exang',
                'oldpeak',
                'slope',
                'ca',
                'thal',
                'num']

    heart = pd.read_csv(
        "~/Google Drive/Discrétisation ICLR19/opendata/heart.dat",
        sep="\s",
        names = column_names,
        header= None,
        index_col = False,
        na_values=[
            '-NaN', '-nan',
            'N/A', 'NA', 'NULL', 'NaN', 'n/a', 'nan', 'null',
            '.'
        ])
    
    heart.num = heart.num-1
    heart.dropna(inplace=True)
    heart.reset_index(inplace=True, drop=True)
    lignes = np.where(heart['restecg'].astype(str)=='1.0')
    heart.drop(heart.index[lignes], inplace=True)

    heart.reset_index(inplace=True, drop=True)
    heart_label_encoders = []

    heart_encoded = heart.copy()

    for j in ['cp', 'restecg', 'slope', 'thal']:
        temp = sk.preprocessing.LabelEncoder()
        temp.fit(heart[j].astype(str))
        heart_label_encoders.append(temp)
        heart_encoded[j] = temp.transform(heart[j].astype(str))
        
    heart_one_hot_encoder = sk.preprocessing.OneHotEncoder(categories='auto',sparse=False,handle_unknown="ignore")
    heart_one_hot_encoder.fit(heart_encoded[[
    'cp', 'restecg', 'slope', 'thal'
    ]])
    heart_one_hot_encoded = heart_encoded.copy()
    heart_one_hot_encoded.drop(
    ['cp', 'restecg', 'slope', 'thal'],
    axis=1,
    inplace=True)
    heart_one_hot_encoded = pd.concat(
    [
    heart_one_hot_encoded,
    pd.DataFrame(
    heart_one_hot_encoder.transform(heart_encoded[[
      'cp', 'restecg', 'slope', 'thal'
    ]]),
    index=heart_one_hot_encoded.index)
    ],
    axis=1)  
    heart_features_train, heart_features_test, heart_perf_train, heart_perf_test = sk.model_selection.train_test_split(
    heart_one_hot_encoded.drop('num', axis=1),
    heart_one_hot_encoded.num,
    test_size=0.33,
    random_state=1)
    heart_nn_features_train = heart_encoded.iloc[
    heart_features_train.index, :].drop(
    'num', axis=1)
    heart_nn_features_test = heart_encoded.iloc[heart_features_test.index, :].drop(
    'num', axis=1)
    heart_nn_perf_train = heart_encoded.iloc[
    heart_features_train.index, :].num
    heart_nn_perf_test = heart_encoded.iloc[heart_features_test.index, :].num

    
    global n
    global n_test
    n = heart_nn_features_train.shape[0]
    n_test = heart_nn_features_test.shape[0]



    x_quant = heart_nn_features_train[[
    'age','sex','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','ca'
    ]].values
    x_qual = heart_nn_features_train[['cp', 'restecg', 'slope', 'thal']].values
    y = heart_nn_perf_train.values


    d1 = x_quant.shape[1]
    d2 = x_qual.shape[1]





    x_quant_test = heart_nn_features_test[[
    'age','sex','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','ca'
    ]].values
    x_qual_test = heart_nn_features_test[['cp', 'restecg', 'slope', 'thal']].values
    y_test = heart_nn_perf_test.values


    for j in range(d2):
        liste_absents = [
        item for item in np.unique(x_qual_test[:, j])
        if item not in np.unique(x_qual[:, j])
        ]
        liste_lignes = [
        np.where(x_qual_test[:, j] == item) for item in liste_absents
        ]
        x_qual_test = np.delete(
        x_qual_test,
        list(chain.from_iterable(list(chain.from_iterable(liste_lignes)))),
        axis=0)
        x_quant_test = np.delete(
        x_quant_test,
        list(chain.from_iterable(list(chain.from_iterable(liste_lignes)))),
        axis=0)
        y_test = np.delete(
        y_test,
        list(chain.from_iterable(list(chain.from_iterable(liste_lignes)))),
        axis=0)
        
    x_qual_encoded = x_qual.copy()
    x_qual_label_encoders = []
    
    for j in range(d2):
        temp = sk.preprocessing.LabelEncoder()
        temp.fit(x_qual[:,j].astype(str))
        x_qual_label_encoders.append(temp)
        x_qual_encoded[:,j] = temp.transform(x_qual[:,j].astype(str))
        
    x_qual_one_hot_encoder = sk.preprocessing.OneHotEncoder(categories='auto',sparse=False,handle_unknown="ignore")
    
    x_qual_one_hot_encoder.fit(x_qual_encoded)
    
    x_qual_dummy = x_qual_one_hot_encoder.transform(x_qual_encoded)
    
    x_qual = x_qual_encoded
    
    x_qual_test_encoded = x_qual_test.copy()
    
    for j in range(d2):
        x_qual_test_encoded[:,j] = x_qual_label_encoders[j].transform(x_qual_test[:,j].astype(str))
        
    x_qual_test = x_qual_test_encoded

    x_qual_dummy_test = x_qual_one_hot_encoder.transform(x_qual_test)


    return x_quant,x_qual,x_qual_dummy,y,x_quant_test,x_qual_test,x_qual_dummy_test,y_test


In [4]:
x_quant,x_qual,x_qual_dummy,y,x_quant_test,x_qual_test,x_qual_dummy_test,y_test = data()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [5]:
np.savetxt("x_quant.csv", x_quant, delimiter=",")
np.savetxt("x_qual.csv", x_qual, delimiter=",")
np.savetxt("y.csv", y, delimiter=",")
np.savetxt("x_quant_test.csv", x_quant_test, delimiter=",")
np.savetxt("x_qual_test.csv", x_qual_test, delimiter=",")
np.savetxt("y_test.csv", y_test, delimiter=",")

In [4]:
n = 179
n_test = 89
d1 = 10
d2 = 4

liste_layers_quant = [None] * d1
liste_layers_qual = [None] * d2
liste_qual_arrays = [None] * d2
liste_qual_arrays_test = [None] * d2

def create_model(x_quant,x_qual,x_qual_dummy,y,x_quant_test,x_qual_test,x_qual_dummy_test,y_test):    
    def initialize_neural_net(m_quant,m_qual):
        liste_inputs_quant = [None] * d1
        liste_inputs_qual = [None] * d2

        liste_layers_quant = [None] * d1
        liste_layers_qual = [None] * d2

        liste_layers_quant_inputs = [None] * d1
        liste_layers_qual_inputs = [None] * d2

        for i in range(d1):
            liste_inputs_quant[i] = Input((1, ))
            liste_layers_quant[i] = Dense(m_quant[i], activation='softmax')
            liste_layers_quant_inputs[i] = liste_layers_quant[i](
                liste_inputs_quant[i])

        for i in range(d2):
            liste_inputs_qual[i] = Input((len(np.unique(x_qual[:, i])), ))
            if (len(np.unique(x_qual[:, i])) > m_qual[i]):
                liste_layers_qual[i] = Dense(
                m_qual[i], activation='softmax', use_bias=False)
            else:
                liste_layers_qual[i] = Dense(
                len(np.unique(x_qual[:, i])), activation='softmax', use_bias=False)

            liste_layers_qual_inputs[i] = liste_layers_qual[i](
                liste_inputs_qual[i])

        return ([
            liste_inputs_quant, liste_layers_quant, liste_layers_quant_inputs,
            liste_inputs_qual, liste_layers_qual, liste_layers_qual_inputs
        ])
    
    
    def from_layers_to_proba_training(d1,d2,liste_layers_quant,liste_layers_qual):

        results = [None] * (d1 + d2)

        for j in range(d1):
            results[j] = K.function([liste_layers_quant[j].input],
                                    [liste_layers_quant[j].output])(
                                        [x_quant[:, j, np.newaxis]])

        for j in range(d2):
            results[j + d1] = K.function([liste_layers_qual[j].input],
                                         [liste_layers_qual[j].output])(
                                             [liste_qual_arrays[j]])

        return (results)
    
    
    
    
    def from_weights_to_proba_test(d1,d2,m_quant,m_qual,history,x_quant_test,x_qual_test,n_test):

        results = [None] * (d1 + d2)    

        for j in range(d1):
            results[j] = np.zeros((n_test,m_quant[j]))
            for i in range(m_quant[j]):
                results[j][:,i] = history.best_weights[j][1][i] + history.best_weights[j][0][0][i]*x_quant_test[:,j]


        for j in range(d2):
            results[j+d1] = np.zeros((n_test,history.best_weights[j+d1][0].shape[1]))
            for i in range(history.best_weights[j+d1][0].shape[1]):
                for k in range(n_test):
                    results[j+d1][k,i] = history.best_weights[j+d1][0][x_qual_test[k,j],i]

        return(results)
    
    
    def evaluate_disc(type,d1,d2,misc):
        if type=="train":
            proba = from_layers_to_proba_training(d1,d2,misc[0],misc[1])
        else:
            proba = from_weights_to_proba_test(d1,d2,misc[0],misc[1],misc[2],misc[3],misc[4],misc[5])


        results = [None] * (d1 + d2)

        if type=="train":
            X_transformed = np.ones((n, 1))
        else:
            X_transformed = np.ones((n_test, 1))

        for j in range(d1 + d2):
            if type=="train":
                results[j] = np.argmax(proba[j][0], axis=1)
            else:
                results[j] = np.argmax(proba[j], axis=1)
            X_transformed = np.concatenate(
                (X_transformed, sk.preprocessing.OneHotEncoder(categories='auto',sparse=False,handle_unknown="ignore").fit_transform(
                    X=results[j].reshape(-1, 1))),
                axis=1)

        proposed_logistic_regression = sk.linear_model.LogisticRegression(
            fit_intercept=False, solver = "lbfgs", C=1e20, tol=1e-8, max_iter=50)


        if type=="train":
            proposed_logistic_regression.fit(X=X_transformed, y=y.reshape((n, )))
            performance = 2 * sk.metrics.log_loss(
              y,
              proposed_logistic_regression.predict_proba(X=X_transformed)[:, 1],
              normalize=False
          ) + proposed_logistic_regression.coef_.shape[1] * np.log(n)
            predicted = proposed_logistic_regression.predict_proba(X_transformed)[:,1]

        else:
            proposed_logistic_regression.fit(X=X_transformed, y=y_test.reshape((n_test, )))
            performance = 2*sk.metrics.roc_auc_score(y_test,proposed_logistic_regression.predict_proba(X_transformed)[:,1])-1
            predicted = proposed_logistic_regression.predict_proba(X_transformed)[:,1]

        return (performance, predicted)


    
    class LossHistory(Callback):
        def on_train_begin(self, logs={}):
            self.losses = []
            self.best_criterion = float("inf")
            self.best_outputs = []

        def on_epoch_end(self, batch, logs={}):
            self.losses.append(evaluate_disc("train",d1,d2,[liste_layers_quant,liste_layers_qual])[0])
            if self.losses[-1] < self.best_criterion:
                self.best_weights = []
                self.best_outputs = []
                self.best_criterion = self.losses[-1]
                for j in range(d1):
                    self.best_weights.append(liste_layers_quant[j].get_weights())
                    self.best_outputs.append(
                        K.function([liste_layers_quant[j].input],
                                   [liste_layers_quant[j].output])(
                                       [x_quant[:, j, np.newaxis]]))
                for j in range(d2):
                    self.best_weights.append(liste_layers_qual[j].get_weights())
                    self.best_outputs.append(
                        K.function([liste_layers_qual[j].input],
                                   [liste_layers_qual[j].output])(
                                       [liste_qual_arrays[j]]))
    
    
    
    essai = {{uniform(4,9)}}

    m_quant = [int(essai)] * d1
    m_qual = [int(essai)] * d2

    global liste_qual_arrays
    liste_qual_arrays = [None] * d2
    cursor = 0
    for j in range(d2):
        liste_qual_arrays[j] = x_qual_dummy[:, cursor:(
            cursor + len(np.unique(x_qual[:, j])))]
        cursor += len(np.unique(x_qual[:, j]))

    global liste_qual_arrays_test
    liste_qual_arrays_test = [None] * d2
    cursor = 0
    for j in range(d2):
        liste_qual_arrays_test[j] = x_qual_dummy_test[:, cursor:(
            cursor + len(np.unique(x_qual_test[:, j])))]
        cursor += len(np.unique(x_qual_test[:, j]))
  
  
    liste_inputs_quant = [None] * d1
    liste_inputs_qual = [None] * d2

    global liste_layers_quant
    global liste_layers_qual

    liste_layers_quant = [None] * d1
    liste_layers_qual = [None] * d2

    liste_layers_quant_inputs = [None] * d1
    liste_layers_qual_inputs = [None] * d2

    for i in range(d1):
        liste_inputs_quant[i] = Input((1, ))
        liste_layers_quant[i] = Dense(m_quant[i], activation='softmax')
        liste_layers_quant_inputs[i] = liste_layers_quant[i](
          liste_inputs_quant[i])

    for i in range(d2):
        liste_inputs_qual[i] = Input((len(np.unique(x_qual[:, i])), ))
        if (len(np.unique(x_qual[:, i])) > m_qual[i]):
            liste_layers_qual[i] = Dense(
            m_qual[i], activation='softmax', use_bias=False)
        else:
            liste_layers_qual[i] = Dense(
            len(np.unique(x_qual[:, i])), activation='softmax', use_bias=False)

        liste_layers_qual_inputs[i] = liste_layers_qual[i](
        liste_inputs_qual[i])


    full_hidden = concatenate(
        list(
          chain.from_iterable(
              [liste_layers_quant_inputs, liste_layers_qual_inputs])))
    output = Dense(1, activation='sigmoid')(full_hidden)
    model = Model(
      inputs=list(chain.from_iterable([liste_inputs_quant, liste_inputs_qual])),
      outputs=[output])  

    
    adam = optimizers.Adam(lr={{choice([10**-3, 10**-2, 10**-1])}})
    rmsprop = optimizers.RMSprop(lr={{choice([10**-3, 10**-2, 10**-1])}})
    sgd = optimizers.SGD(lr={{choice([10**-3, 10**-2, 10**-1])}})
   
    choiceval = {{choice(['adam', 'sgd', 'rmsprop'])}}
    if choiceval == 'adam':
        optim = adam
    elif choiceval == 'rmsprop':
        optim = rmsprop
    else:
        optim = sgd
        
 
    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])

    history = LossHistory()

    callbacks = [
      ReduceLROnPlateau(
          monitor='loss',
          factor=0.5,
          patience=10,
          verbose=0,
          mode='auto',
          min_delta=0.0001,
          cooldown=0,
          min_lr=0), history
    ]
    model.fit(
      list(chain.from_iterable([list(x_quant.T), liste_qual_arrays])),
      y,
      epochs=300,
      batch_size={{choice([32,64,128])}},
      verbose=1,
      callbacks=callbacks)

    global n_test
    n_test = x_quant_test.shape[0]
    performance, predicted = evaluate_disc("test",d1,d2,misc=[m_quant,m_qual,history,x_quant_test,x_qual_test,n_test])
  
    K.clear_session()
    return {'loss': -performance, 'status': STATUS_OK, 'model': model, 'predicted': predicted}


In [5]:
trials = Trials()

best_run, best_model, space = optim.minimize(model=create_model,
                          data=data,
                          algo=tpe.suggest,
                          max_evals=10,
                          trials=trials,
                          notebook_name='hyperas - heart - dump - ICML19',
                          eval_space=True,
                          return_space=True
)

>>> Imports:
#coding=utf-8

try:
    from itertools import chain
except:
    pass

try:
    import random
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import sklearn as sk
except:
    pass

try:
    import sklearn.metrics
except:
    pass

try:
    import scipy as sp
except:
    pass

try:
    import sklearn.linear_model
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from keras import *
except:
    pass

try:
    from keras.layers import *
except:
    pass

try:
    from keras.callbacks import LambdaCallback, Callback, ReduceLROnPlateau, TensorBoard
except:
    pass

try:
    import sklearn.linear_model
except:
    pass

try:
    from hyperas.distributions import uniform, choice
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
 

/Users/adrien/Google Drive/Colab Notebooks/temp_model.py:133: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  '.'


Epoch 1/300
179/179 [==============================] - 1s 7ms/step - loss: 0.6823 - acc: 0.5419
Epoch 2/300
179/179 [==============================] - 0s 70us/step - loss: 0.6659 - acc: 0.5978
Epoch 3/300
179/179 [==============================] - 0s 78us/step - loss: 0.6615 - acc: 0.5866
Epoch 4/300
179/179 [==============================] - 0s 182us/step - loss: 0.6589 - acc: 0.5866
Epoch 5/300
179/179 [==============================] - 0s 84us/step - loss: 0.6565 - acc: 0.5866
Epoch 6/300
179/179 [==============================] - 0s 76us/step - loss: 0.6544 - acc: 0.5866
Epoch 7/300
179/179 [==============================] - 0s 140us/step - loss: 0.6532 - acc: 0.5866
Epoch 8/300
179/179 [==============================] - 0s 81us/step - loss: 0.6531 - acc: 0.5978
Epoch 9/300
179/179 [==============================] - 0s 145us/step - loss: 0.6486 - acc: 0.5866
Epoch 10/300
179/179 [==============================] - 0s 72us/step - loss: 0.6466 - acc: 0.5866
Epoch 11/300
179/179 [=====

179/179 [==============================] - 0s 93us/step - loss: 0.4257 - acc: 0.8101
Epoch 167/300
179/179 [==============================] - 0s 70us/step - loss: 0.4241 - acc: 0.8156
Epoch 168/300
179/179 [==============================] - 0s 101us/step - loss: 0.4242 - acc: 0.8212
Epoch 169/300
179/179 [==============================] - 0s 77us/step - loss: 0.4226 - acc: 0.8156
Epoch 170/300
179/179 [==============================] - 0s 87us/step - loss: 0.4217 - acc: 0.8156
Epoch 171/300
179/179 [==============================] - 0s 68us/step - loss: 0.4207 - acc: 0.8156
Epoch 172/300
179/179 [==============================] - 0s 104us/step - loss: 0.4200 - acc: 0.8212
Epoch 173/300
179/179 [==============================] - 0s 92us/step - loss: 0.4191 - acc: 0.8212
Epoch 174/300
179/179 [==============================] - 0s 73us/step - loss: 0.4188 - acc: 0.8212
Epoch 175/300
179/179 [==============================] - 0s 78us/step - loss: 0.4179 - acc: 0.8212
Epoch 176/300
179/179 

179/179 [==============================] - 0s 57us/step - loss: 0.7379 - acc: 0.4134
Epoch 32/300
179/179 [==============================] - 0s 107us/step - loss: 0.7373 - acc: 0.4134
Epoch 33/300
179/179 [==============================] - 0s 88us/step - loss: 0.7368 - acc: 0.4134
Epoch 34/300
179/179 [==============================] - 0s 61us/step - loss: 0.7363 - acc: 0.4134
Epoch 35/300
179/179 [==============================] - 0s 53us/step - loss: 0.7358 - acc: 0.4134
Epoch 36/300
179/179 [==============================] - 0s 79us/step - loss: 0.7353 - acc: 0.4134
Epoch 37/300
179/179 [==============================] - 0s 54us/step - loss: 0.7348 - acc: 0.4134
Epoch 38/300
179/179 [==============================] - 0s 48us/step - loss: 0.7343 - acc: 0.4134
Epoch 39/300
179/179 [==============================] - 0s 47us/step - loss: 0.7337 - acc: 0.4134
Epoch 40/300
179/179 [==============================] - 0s 76us/step - loss: 0.7333 - acc: 0.4134
Epoch 41/300
179/179 [==========

...

179/179 [==============================] - 0s 129us/step - loss: 0.2842 - acc: 0.8883
Epoch 210/300
179/179 [==============================] - 0s 108us/step - loss: 0.2842 - acc: 0.8883
Epoch 211/300
179/179 [==============================] - 0s 103us/step - loss: 0.2842 - acc: 0.8883
Epoch 212/300
179/179 [==============================] - 0s 120us/step - loss: 0.2842 - acc: 0.8883
Epoch 213/300
179/179 [==============================] - 0s 106us/step - loss: 0.2842 - acc: 0.8883
Epoch 214/300
179/179 [==============================] - 0s 128us/step - loss: 0.2842 - acc: 0.8883
Epoch 215/300
179/179 [==============================] - 0s 118us/step - loss: 0.2842 - acc: 0.8883
Epoch 216/300
179/179 [==============================] - 0s 101us/step - loss: 0.2842 - acc: 0.8883
Epoch 217/300
179/179 [==============================] - 0s 93us/step - loss: 0.2842 - acc: 0.8883
Epoch 218/300
179/179 [==============================] - 0s 91us/step - loss: 0.2841 - acc: 0.8883
Epoch 219/300
17

179/179 [==============================] - 0s 107us/step - loss: 0.2870 - acc: 0.8994
Epoch 75/300
179/179 [==============================] - 0s 87us/step - loss: 0.2864 - acc: 0.8994
Epoch 76/300
179/179 [==============================] - 0s 89us/step - loss: 0.2859 - acc: 0.8994
Epoch 77/300
179/179 [==============================] - 0s 110us/step - loss: 0.2859 - acc: 0.8994
Epoch 78/300
179/179 [==============================] - 0s 90us/step - loss: 0.2865 - acc: 0.8994
Epoch 79/300
179/179 [==============================] - 0s 86us/step - loss: 0.2863 - acc: 0.8994
Epoch 80/300
179/179 [==============================] - 0s 85us/step - loss: 0.2862 - acc: 0.8994
Epoch 81/300
179/179 [==============================] - 0s 87us/step - loss: 0.2868 - acc: 0.8994
Epoch 82/300
179/179 [==============================] - 0s 90us/step - loss: 0.2861 - acc: 0.8994
Epoch 83/300
179/179 [==============================] - 0s 89us/step - loss: 0.2858 - acc: 0.8994
Epoch 84/300
179/179 [=========

179/179 [==============================] - 0s 94us/step - loss: 0.2855 - acc: 0.8994
Epoch 239/300
179/179 [==============================] - 0s 92us/step - loss: 0.2855 - acc: 0.8994
Epoch 240/300
179/179 [==============================] - 0s 105us/step - loss: 0.2855 - acc: 0.8994
Epoch 241/300
179/179 [==============================] - 0s 88us/step - loss: 0.2855 - acc: 0.8994
Epoch 242/300
179/179 [==============================] - 0s 117us/step - loss: 0.2855 - acc: 0.8994
Epoch 243/300
179/179 [==============================] - 0s 88us/step - loss: 0.2855 - acc: 0.8994
Epoch 244/300
179/179 [==============================] - 0s 112us/step - loss: 0.2855 - acc: 0.8994
Epoch 245/300
179/179 [==============================] - 0s 93us/step - loss: 0.2855 - acc: 0.8994
Epoch 246/300
179/179 [==============================] - 0s 136us/step - loss: 0.2855 - acc: 0.8994
Epoch 247/300
179/179 [==============================] - 0s 82us/step - loss: 0.2855 - acc: 0.8994
Epoch 248/300
179/17

179/179 [==============================] - 0s 87us/step - loss: 0.3012 - acc: 0.8827
Epoch 104/300
179/179 [==============================] - 0s 82us/step - loss: 0.3010 - acc: 0.8827
Epoch 105/300
179/179 [==============================] - 0s 103us/step - loss: 0.3000 - acc: 0.8883
Epoch 106/300
179/179 [==============================] - 0s 80us/step - loss: 0.2996 - acc: 0.8883
Epoch 107/300
179/179 [==============================] - 0s 79us/step - loss: 0.2992 - acc: 0.8883
Epoch 108/300
179/179 [==============================] - 0s 82us/step - loss: 0.2989 - acc: 0.8883
Epoch 109/300
179/179 [==============================] - 0s 90us/step - loss: 0.2985 - acc: 0.8883
Epoch 110/300
179/179 [==============================] - 0s 88us/step - loss: 0.2983 - acc: 0.8883
Epoch 111/300
179/179 [==============================] - 0s 96us/step - loss: 0.2980 - acc: 0.8883
Epoch 112/300
179/179 [==============================] - 0s 77us/step - loss: 0.2975 - acc: 0.8883
Epoch 113/300
179/179 [

179/179 [==============================] - 0s 104us/step - loss: 0.2847 - acc: 0.8883
Epoch 268/300
179/179 [==============================] - 0s 91us/step - loss: 0.2847 - acc: 0.8883
Epoch 269/300
179/179 [==============================] - 0s 246us/step - loss: 0.2846 - acc: 0.8883
Epoch 270/300
179/179 [==============================] - 0s 91us/step - loss: 0.2848 - acc: 0.8883
Epoch 271/300
179/179 [==============================] - 0s 72us/step - loss: 0.2847 - acc: 0.8883
Epoch 272/300
179/179 [==============================] - 0s 92us/step - loss: 0.2847 - acc: 0.8883
Epoch 273/300
179/179 [==============================] - 0s 277us/step - loss: 0.2847 - acc: 0.8883
Epoch 274/300
179/179 [==============================] - 0s 83us/step - loss: 0.2847 - acc: 0.8883
Epoch 275/300
179/179 [==============================] - 0s 78us/step - loss: 0.2847 - acc: 0.8883
Epoch 276/300
179/179 [==============================] - 0s 301us/step - loss: 0.2846 - acc: 0.8883
Epoch 277/300
179/17

In [6]:
print(best_run)

{'batch_size': 64, 'choiceval': 'sgd', 'essai': 7.462695171578595, 'lr': 0.001, 'lr_1': 0.01, 'lr_2': 0.1}


In [7]:
trials.best_trial

{'state': 2,
 'tid': 0,
 'spec': None,
 'result': {'loss': -0.8626262626262626,
  'status': 'ok',
  'model': <keras.engine.training.Model at 0x1a2a1ef390>,
  'predicted': array([0.04991082, 0.03322218, 0.89832356, 0.99247994, 0.99434962,
         0.36376293, 0.84710767, 0.89903432, 0.06107192, 0.08706849,
         0.34861298, 0.27379132, 0.27379132, 0.00392652, 0.97628872,
         0.98447385, 0.88804744, 0.00392652, 0.41643953, 0.88804744,
         0.10642328, 0.06228042, 0.85350153, 0.03322218, 0.9232504 ,
         0.33453681, 0.83670811, 0.38859493, 0.41845838, 0.89518454,
         0.06107192, 0.92380169, 0.27379132, 0.03322218, 0.00797134,
         0.03322218, 0.4948783 , 0.78960194, 0.3328013 , 0.99247994,
         0.06939704, 0.99247994, 0.33453681, 0.84820872, 0.04991082,
         0.98447385, 0.83557069, 0.85350153, 0.09044169, 0.05452374,
         0.36184376, 0.00797134, 0.99232246, 0.23409841, 0.4215231 ,
         0.29110746, 0.86249997, 0.00392652, 0.89832356, 0.39056914,
   

In [8]:
trials.results

[{'loss': -0.8626262626262626,
  'status': 'ok',
  'model': <keras.engine.training.Model at 0x1a2a1ef390>,
  'predicted': array([0.04991082, 0.03322218, 0.89832356, 0.99247994, 0.99434962,
         0.36376293, 0.84710767, 0.89903432, 0.06107192, 0.08706849,
         0.34861298, 0.27379132, 0.27379132, 0.00392652, 0.97628872,
         0.98447385, 0.88804744, 0.00392652, 0.41643953, 0.88804744,
         0.10642328, 0.06228042, 0.85350153, 0.03322218, 0.9232504 ,
         0.33453681, 0.83670811, 0.38859493, 0.41845838, 0.89518454,
         0.06107192, 0.92380169, 0.27379132, 0.03322218, 0.00797134,
         0.03322218, 0.4948783 , 0.78960194, 0.3328013 , 0.99247994,
         0.06939704, 0.99247994, 0.33453681, 0.84820872, 0.04991082,
         0.98447385, 0.83557069, 0.85350153, 0.09044169, 0.05452374,
         0.36184376, 0.00797134, 0.99232246, 0.23409841, 0.4215231 ,
         0.29110746, 0.86249997, 0.00392652, 0.89832356, 0.39056914,
         0.0363517 , 0.99247994, 0.4948783 , 0.87463

In [9]:
trials.best_trial['result']['predicted']

array([0.04991082, 0.03322218, 0.89832356, 0.99247994, 0.99434962,
       0.36376293, 0.84710767, 0.89903432, 0.06107192, 0.08706849,
       0.34861298, 0.27379132, 0.27379132, 0.00392652, 0.97628872,
       0.98447385, 0.88804744, 0.00392652, 0.41643953, 0.88804744,
       0.10642328, 0.06228042, 0.85350153, 0.03322218, 0.9232504 ,
       0.33453681, 0.83670811, 0.38859493, 0.41845838, 0.89518454,
       0.06107192, 0.92380169, 0.27379132, 0.03322218, 0.00797134,
       0.03322218, 0.4948783 , 0.78960194, 0.3328013 , 0.99247994,
       0.06939704, 0.99247994, 0.33453681, 0.84820872, 0.04991082,
       0.98447385, 0.83557069, 0.85350153, 0.09044169, 0.05452374,
       0.36184376, 0.00797134, 0.99232246, 0.23409841, 0.4215231 ,
       0.29110746, 0.86249997, 0.00392652, 0.89832356, 0.39056914,
       0.0363517 , 0.99247994, 0.4948783 , 0.8746324 , 0.04991082,
       0.98935858, 0.00392652, 0.43263477, 0.85350153, 0.09044169,
       0.08043692, 0.43263477, 0.80408164, 0.93106517, 0.98854

In [10]:
x_quant,x_qual,x_qual_dummy,y,x_quant_test,x_qual_test,x_qual_dummy_test,y_test=data()


alpha = .95
y_pred = trials.best_trial['result']['predicted']
y_true = y_test

auc, auc_cov = delong_roc_variance(
    y_true,
    y_pred)

auc_std = np.sqrt(auc_cov)
lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)

ci = stats.norm.ppf(
    lower_upper_q,
    loc=auc,
    scale=auc_std)

ci[ci > 1] = 1

print('Gini:', 2*auc-1)
print('AUC COV:', auc_cov)
print('95% Gini CI:', 2*ci-1)

Gini: 0.8626262626262626
AUC COV: 0.00072525983779107
95% Gini CI: [0.75706006 0.96819247]


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
